# Encoding

In [2]:
cd ..

/home/simonchuth/Desktop/git_repo/patentAI


In [1]:
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import tensorflow as tf

In [3]:
encoder_path = 'encoder/use4_model/'
encoder = hub.load(encoder_path)

In [4]:
import pickle
pkl_path = 'pkl_files/sample_data/ipos_data.pkl'
with open(pkl_path, 'rb') as pkl_file:
    dataset = pickle.load(pkl_file)

In [6]:
import random

In [7]:
random.shuffle(dataset)

In [10]:
test_ratio = 0.1
test_size = int(len(dataset) * test_ratio)
test_set = dataset[:test_size]
train_set = dataset[test_size:]

In [18]:
from tqdm import tqdm
import re
from tensorflow.keras.preprocessing.text import text_to_word_sequence

term_pattern = r'".+?"'

In [49]:
end_word = np.zeros((1,512))
end_tensor = tf.convert_to_tensor(end_word)
end_tensor = tf.cast(end_tensor, tf.float32)
end_tensor

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

In [50]:
context_tensor = []
definition_tensor = []
terms = []
for app in tqdm(train_set):
    intro = app[0]
    claims = ' '.join(app[1])
    context = [intro, claims]
    context = encoder(context)
    context = tf.reshape(context, [1024])
    context_tensor.append(context)
    
    definition = app[2]
    term = re.findall(term_pattern, definition)[0]
    terms.append(term)
    term_tensor = encoder([term])
    definition_tokens = text_to_word_sequence(definition)
    definition_tokens = encoder(definition_tokens)
    definition_tokens = tf.concat([definition_tokens, end_tensor], axis = 0)
    definition_tensor.append(definition_tokens)

100%|██████████| 12132/12132 [04:04<00:00, 49.65it/s]


In [52]:
train_pkl = 'pkl_files/sample_data/train_tensor.pkl'

combine = [context_tensor, definition_tensor]
with open(train_pkl, 'wb') as pkl_file:
    pickle.dump(combine, pkl_file)

In [54]:
context_tensor = []
definition_tensor = []
terms = []
for app in tqdm(test_set):
    intro = app[0]
    claims = ' '.join(app[1])
    context = [intro, claims]
    context = encoder(context)
    context = tf.reshape(context, [1024])
    context_tensor.append(context)
    
    definition = app[2]
    term = re.findall(term_pattern, definition)[0]
    terms.append(term)
    term_tensor = encoder([term])
    definition_tokens = text_to_word_sequence(definition)
    definition_tokens = encoder(definition_tokens)
    definition_tokens = tf.concat([definition_tokens, end_tensor], axis = 0)
    definition_tensor.append(definition_tokens)
    
test_pkl = 'pkl_files/sample_data/test_tensor.pkl'

combine = [context_tensor, definition_tensor]
with open(test_pkl, 'wb') as pkl_file:
    pickle.dump(combine, pkl_file)

100%|██████████| 1347/1347 [00:26<00:00, 49.94it/s]


# Combine data

In [1]:
cd ..

In [2]:
import pickle
pkl_path = 'pkl_files/sample_data/2_train_tensor.pkl'
with open(pkl_path, 'rb') as pkl_file:
    data = pickle.load(pkl_file)

In [3]:
context_tensor, target_tensor, terms = data

In [4]:
import tensorflow as tf
context = tf.concat(context_tensor, axis=0)
target = tf.concat(target_tensor, axis=0)

In [5]:
data_tensor = [context, target]

pkl_path = 'pkl_files/sample_data/2_train_data.pkl'
with open(pkl_path, 'wb') as pkl_file:
    pickle.dump(data_tensor, pkl_file)

# Model building

In [1]:
cd ..

C:\Users\simon\OneDrive\Desktop\git_repo\patentAI


In [2]:
import pickle
pkl_path = 'pkl_files/sample_data/1_train_data.pkl'
with open(pkl_path, 'rb') as pkl_file:
    data1 = pickle.load(pkl_file)
context_1, target_1 = data1

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, CosineSimilarity
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [7]:
params = {'layers': [1024, 1024, 512, 512],
          'input_dim': 2048,
          'output_dim': 512,
          'optimizer': Adam(),
          'loss': CosineSimilarity(),
          'metrics': ['cosine_similarity'],
          'batch_size': 10000,
          'epochs': 100}

In [5]:
model = Sequential()

for i, layer in enumerate(params['layers']):
    if i == 0:
        model.add(Dense(layer,
                        input_dim=params['input_dim'],
                        activation='relu',
                        name='Input'))
    else:
        model.add(Dense(layer,
                        activation='relu',
                        name=f'Hidden_{i}'))
        
model.add(Dense(params['output_dim'],
                activation='relu',
                name='Output'))

model.compile(optimizer=params['optimizer'],
              loss=params['loss'],
              metrics=params['metrics'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (Dense)                (None, 1024)              2098176   
_________________________________________________________________
Hidden_1 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
Hidden_2 (Dense)             (None, 512)               524800    
_________________________________________________________________
Hidden_3 (Dense)             (None, 512)               262656    
_________________________________________________________________
Output (Dense)               (None, 512)               262656    
Total params: 4,197,888
Trainable params: 4,197,888
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(context_1, target_1,
          batch_size=params['batch_size'],
          epochs=params['epochs'])

Train on 172695 samples
Epoch 1/100
172695/172695 [==============================] - 3s 17us/sample - loss: -0.3245 - cosine_similarity: 0.3245
Epoch 2/100
172695/172695 [==============================] - 2s 10us/sample - loss: -0.3725 - cosine_similarity: 0.3725
Epoch 3/100
172695/172695 [==============================] - 2s 10us/sample - loss: -0.3742 - cosine_similarity: 0.3742
Epoch 4/100
172695/172695 [==============================] - 2s 10us/sample - loss: -0.3744 - cosine_similarity: 0.3744
Epoch 5/100
172695/172695 [==============================] - 2s 10us/sample - loss: -0.3745 - cosine_similarity: 0.3745
Epoch 6/100
172695/172695 [==============================] - 2s 10us/sample - loss: -0.3745 - cosine_similarity: 0.3745
Epoch 7/100
172695/172695 [==============================] - 2s 10us/sample - loss: -0.3746 - cosine_similarity: 0.3746
Epoch 8/100
172695/172695 [==============================] - 2s 10us/sample - loss: -0.3749 - cosine_similarity: 0.3749
Epoch 9/100
1726

172695/172695 [==============================] - 1s 9us/sample - loss: -0.4785 - cosine_similarity: 0.4785
Epoch 65/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.4836 - cosine_similarity: 0.4836
Epoch 66/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.4893 - cosine_similarity: 0.4893
Epoch 67/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.4938 - cosine_similarity: 0.4938
Epoch 68/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.4891 - cosine_similarity: 0.4891
Epoch 69/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.4942 - cosine_similarity: 0.4942
Epoch 70/100
172695/172695 [==============================] - 1s 8us/sample - loss: -0.4976 - cosine_similarity: 0.4976
Epoch 71/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.4965 - cosine_similarity: 0.4965
Epoch 72/100
172695/172695 [=========================

In [8]:
model.save('models/model_on_data1.h5')

In [4]:
model = load_model('models/model_on_data1.h5')

In [4]:
import pickle
pkl_path = 'pkl_files/sample_data/2_train_data.pkl'
with open(pkl_path, 'rb') as pkl_file:
    data2 = pickle.load(pkl_file)
context_2, target_2 = data2

In [7]:
model.fit(context_2, target_2,
          batch_size=params['batch_size'],
          epochs=params['epochs'])

Train on 165829 samples
Epoch 1/100
165829/165829 [==============================] - 2s 12us/sample - loss: -0.4125 - cosine_similarity: 0.4125
Epoch 2/100
165829/165829 [==============================] - 1s 8us/sample - loss: -0.4274 - cosine_similarity: 0.4274
Epoch 3/100
165829/165829 [==============================] - 1s 8us/sample - loss: -0.4379 - cosine_similarity: 0.4379
Epoch 4/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.4480 - cosine_similarity: 0.4480
Epoch 5/100
165829/165829 [==============================] - 1s 8us/sample - loss: -0.4569 - cosine_similarity: 0.4569
Epoch 6/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.4645 - cosine_similarity: 0.4645
Epoch 7/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.4714 - cosine_similarity: 0.4714
Epoch 8/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.4768 - cosine_similarity: 0.4768
Epoch 9/100
165829/1658

165829/165829 [==============================] - 1s 9us/sample - loss: -0.5583 - cosine_similarity: 0.5583
Epoch 67/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.5578 - cosine_similarity: 0.5578
Epoch 68/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.5570 - cosine_similarity: 0.5570
Epoch 69/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.5586 - cosine_similarity: 0.5586
Epoch 70/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.5598 - cosine_similarity: 0.5598
Epoch 71/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.5608 - cosine_similarity: 0.5608
Epoch 72/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.5621 - cosine_similarity: 0.5621
Epoch 73/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.5627 - cosine_similarity: 0.56270s - loss: -0.5637 - cosine_simila
Epoch 74/100
165829

In [8]:
model.save('models/model_on_data12.h5')

In [5]:
model = load_model('models/model_on_data12.h5')

In [9]:
for i in range(9):
    print('')
    print(i)
    model.fit(context_1, target_1,
              batch_size=params['batch_size'],
              epochs=params['epochs'])
    model.fit(context_2, target_2,
              batch_size=params['batch_size'],
              epochs=params['epochs'])
model.save('models/model_on_data12x20.h5')


0
Train on 172695 samples
Epoch 1/100
172695/172695 [==============================] - 2s 13us/sample - loss: -0.5614 - cosine_similarity: 0.5614
Epoch 2/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5749 - cosine_similarity: 0.5749
Epoch 3/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5825 - cosine_similarity: 0.5825
Epoch 4/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5864 - cosine_similarity: 0.5864
Epoch 5/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5903 - cosine_similarity: 0.5903
Epoch 6/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5925 - cosine_similarity: 0.5925
Epoch 7/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5952 - cosine_similarity: 0.5952
Epoch 8/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5963 - cosine_similarity: 0.5963
Epoch 9/100
172695/1

172695/172695 [==============================] - 1s 9us/sample - loss: -0.6114 - cosine_similarity: 0.6114
Epoch 69/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6116 - cosine_similarity: 0.6116
Epoch 70/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6119 - cosine_similarity: 0.6119
Epoch 71/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6117 - cosine_similarity: 0.6117
Epoch 72/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6111 - cosine_similarity: 0.6111
Epoch 73/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6108 - cosine_similarity: 0.6108
Epoch 74/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.6107 - cosine_similarity: 0.6107
Epoch 75/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6106 - cosine_similarity: 0.6106
Epoch 76/100
172695/172695 [=========================

165829/165829 [==============================] - 1s 9us/sample - loss: -0.6101 - cosine_similarity: 0.6101
Epoch 37/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6104 - cosine_similarity: 0.6104
Epoch 38/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6107 - cosine_similarity: 0.6107
Epoch 39/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6109 - cosine_similarity: 0.6109
Epoch 40/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6108 - cosine_similarity: 0.6108
Epoch 41/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6111 - cosine_similarity: 0.6111
Epoch 42/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6106 - cosine_similarity: 0.6106
Epoch 43/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6101 - cosine_similarity: 0.6101
Epoch 44/100
165829/165829 [=========================

172695/172695 [==============================] - 1s 9us/sample - loss: -0.5787 - cosine_similarity: 0.5787
Epoch 4/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.5821 - cosine_similarity: 0.5821
Epoch 5/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5857 - cosine_similarity: 0.5857
Epoch 6/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5878 - cosine_similarity: 0.5878
Epoch 7/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5921 - cosine_similarity: 0.5921
Epoch 8/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5942 - cosine_similarity: 0.5942
Epoch 9/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5943 - cosine_similarity: 0.5943
Epoch 10/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.5959 - cosine_similarity: 0.5959
Epoch 11/100
172695/172695 [==============================]

172695/172695 [==============================] - 1s 9us/sample - loss: -0.6086 - cosine_similarity: 0.6086
Epoch 72/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6079 - cosine_similarity: 0.6079
Epoch 73/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.6083 - cosine_similarity: 0.6083
Epoch 74/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.6089 - cosine_similarity: 0.6089
Epoch 75/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6087 - cosine_similarity: 0.6087
Epoch 76/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6088 - cosine_similarity: 0.6088
Epoch 77/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6092 - cosine_similarity: 0.6092
Epoch 78/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6087 - cosine_similarity: 0.6087
Epoch 79/100
172695/172695 [=========================

...

172695/172695 [==============================] - 1s 9us/sample - loss: -0.6143 - cosine_similarity: 0.6143
Epoch 80/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6136 - cosine_similarity: 0.6136
Epoch 81/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6129 - cosine_similarity: 0.6129
Epoch 82/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6132 - cosine_similarity: 0.6132
Epoch 83/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6137 - cosine_similarity: 0.6137
Epoch 84/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6137 - cosine_similarity: 0.6137
Epoch 85/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6140 - cosine_similarity: 0.61401s - loss: -0.
Epoch 86/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6133 - cosine_similarity: 0.6133
Epoch 87/100
172695/172695 [===========

165829/165829 [==============================] - 1s 9us/sample - loss: -0.6154 - cosine_similarity: 0.6154
Epoch 47/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6158 - cosine_similarity: 0.6158
Epoch 48/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6161 - cosine_similarity: 0.6161
Epoch 49/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6163 - cosine_similarity: 0.6163
Epoch 50/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6164 - cosine_similarity: 0.6164
Epoch 51/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6169 - cosine_similarity: 0.6169
Epoch 52/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6170 - cosine_similarity: 0.6170
Epoch 53/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6170 - cosine_similarity: 0.61700s - loss: -0.6173 - cosine_simi
Epoch 54/100
165829/1

172695/172695 [==============================] - 1s 9us/sample - loss: -0.6081 - cosine_similarity: 0.6081
Epoch 12/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6090 - cosine_similarity: 0.6090
Epoch 13/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6094 - cosine_similarity: 0.6094
Epoch 14/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6109 - cosine_similarity: 0.6109
Epoch 15/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6106 - cosine_similarity: 0.6106
Epoch 16/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6113 - cosine_similarity: 0.6113
Epoch 17/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6118 - cosine_similarity: 0.6118
Epoch 18/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6121 - cosine_similarity: 0.6121
Epoch 19/100
172695/172695 [=========================

172695/172695 [==============================] - 1s 9us/sample - loss: -0.6159 - cosine_similarity: 0.6159
Epoch 79/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6165 - cosine_similarity: 0.6165
Epoch 80/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6157 - cosine_similarity: 0.6157
Epoch 81/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.6153 - cosine_similarity: 0.6153
Epoch 82/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6152 - cosine_similarity: 0.6152
Epoch 83/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.6150 - cosine_similarity: 0.6150
Epoch 84/100
172695/172695 [==============================] - 1s 9us/sample - loss: -0.6145 - cosine_similarity: 0.6145
Epoch 85/100
172695/172695 [==============================] - 2s 9us/sample - loss: -0.6148 - cosine_similarity: 0.6148
Epoch 86/100
172695/172695 [=========================

165829/165829 [==============================] - 1s 9us/sample - loss: -0.6170 - cosine_similarity: 0.6170
Epoch 46/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6171 - cosine_similarity: 0.6171
Epoch 47/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6173 - cosine_similarity: 0.6173
Epoch 48/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6173 - cosine_similarity: 0.6173
Epoch 49/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6173 - cosine_similarity: 0.6173
Epoch 50/100
165829/165829 [==============================] - 1s 8us/sample - loss: -0.6173 - cosine_similarity: 0.6173
Epoch 51/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6174 - cosine_similarity: 0.6174
Epoch 52/100
165829/165829 [==============================] - 1s 9us/sample - loss: -0.6174 - cosine_similarity: 0.6174
Epoch 53/100
165829/165829 [=========================

# Evaluation

In [14]:
import pickle
pkl_path = 'pkl_files/sample_data/test_tensor.pkl'
with open(pkl_path, 'rb') as pkl_file:
    context_test, target_test, term = pickle.load(pkl_file)

In [15]:
import tensorflow as tf
context_test = tf.concat(context_test, axis=0)
target_test = tf.concat(target_test, axis=0)

In [16]:
model.evaluate(context_test, target_test)

36596/36596 [==============================] - 3s 83us/sample - loss: -0.4063 - cosine_similarity: 0.4063


[-0.40633006755973955, 0.4063302]